In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse

In [20]:
url_base='https://www.vghks.gov.tw'
url_work_table='/News.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

In [21]:
def get_tables_part(soup):
    tables = soup.find_all('tbody')
    if len(tables) == 1:
        return None,tables
    else:
        return tables[1],tables[0]
pages_table,work_tables = get_tables_part(soup)

In [ ]:
print("page list:",pages_table,"work list:",work_tables)

# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [23]:
if pages_table == None:
    print("only one page")
    pages_link=None
else:
    #print(pages_table.find_all('a'))
    pages_link=[]
    pages_link.append(url)
    for i, item in enumerate(pages_table.find_all('a')):
        s=item.get('href')
        link_s = urlparse(url)._replace(path=s,query=[]).geturl()
        if link_s not in pages_link:
            pages_link.append(link_s)
        print('#{} {}'.format(i , s, link_s))

pages_link

#0 /News.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&page=2&PageSize=20
#1 /News.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&page=2&PageSize=20


['https://www.vghks.gov.tw/News.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657',
 'https://www.vghks.gov.tw/News.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&page=2&PageSize=20']

# Get work table

The `get_work_page` function is used for getting each work page

In [6]:
def get_work_page(page_url,page=None):
    global url
    if page != None:
        page_url = page_url+'?page='+str(page)
    if url == None:
        page_url = url
        print(page_url)
    g=requests.get(page_url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

Using `get_work_page` to read each page and using `get_tables_part` read work table

In [24]:
def get_work_table(table_,work_table):
    for i, item in enumerate(table_):
        if item.find('a'):
            s=item.find('a')
            if s == -1:
                s = item
                print(i, s)
            else:
                title = s.get('title')
                link = s.get('href')
                link_s = urlparse(url)._replace(path=link,query=[]).geturl()
                all_td = item.find_all('td')
                origination = all_td[1].find('p').text
                dead_line = all_td[-1].find('p').text
                print(i,title,link_s,origination,dead_line)
                work_table.append([i-2, title, dead_line, link_s ])
    return work_table

In [27]:
work_table=[]
if type(pages_link) != list:
    table_ = work_tables
    work_table = get_work_table(table_,work_table)
else:
    for p_i, p_item in enumerate(pages_link):
        soup_ = get_work_page(p_item)
        a, table_ = get_tables_part(soup_)
        get_work_table(table_,work_table)


0 

1 病理檢驗部師(三)級主治醫師甄選公告 https://www.vghks.gov.tw/News_Content.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&s=32F4D1DF3EE6E498 病理檢驗部 111-05-19
2 

3 藥學部徵契約藥師 https://www.vghks.gov.tw/News_Content.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&s=04305D8A8F81F8A2 藥學部 111-05-13
4 

5 高齡醫學中心徴專任助理（個案管理師） https://www.vghks.gov.tw/News_Content.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&s=D7EA23BCD9B6C0EF 高齡醫學中心 111-05-16
6 

7 高齡醫學中心徴契約醫務管理組員(社區整合型服務中心(A級單位)個案管理員) https://www.vghks.gov.tw/News_Content.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&s=C22C988D6DE43BD6 高齡醫學中心 111-05-13
8 

9 精神部徵高雄榮總屏東分院大武院區招募臨床心理師 https://www.vghks.gov.tw/News_Content.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&s=49D477F4AAAF57F8 精神部 111-05-10
10 

11 屏東大武籌備處-契約行政助理(身心障礙) https://www.vghks.gov.tw/News_Content.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&s=D9E277B4223A35C2 人事室 111-05-09
12 

13 胸腔內科-短期研究助理 https://www.vghks.gov.tw/News_Content.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657&s=2D505D272C86C66D 胸腔內科 1

In [28]:
work_table=pd.DataFrame(work_table, columns=['no','召聘職稱','期限' ,'連結'])

print(work_table)

    no                                   召聘職稱         期限  \
0   -1                     病理檢驗部師(三)級主治醫師甄選公告  111-05-19   
1    1                               藥學部徵契約藥師  111-05-13   
2    3                     高齡醫學中心徴專任助理（個案管理師）  111-05-16   
3    5  高齡醫學中心徴契約醫務管理組員(社區整合型服務中心(A級單位)個案管理員)  111-05-13   
4    7                精神部徵高雄榮總屏東分院大武院區招募臨床心理師  111-05-10   
5    9                   屏東大武籌備處-契約行政助理(身心障礙)  111-05-09   
6   11                            胸腔內科-短期研究助理  111-05-22   
7   13                     急診醫學科-111年度第一年住院醫師  111-05-31   
8   15                        心臟內科研究助理(非院內計畫)  111-06-30   
9   17                           放射線部-契約醫事放射師  111-05-05   
10  19                    麻醉部招訓第七期麻醉護理人員訓練班公告  111-05-13   
11  21          病理檢驗部(屏東大武院區)契約醫事檢驗師(儲備) 甄選公告  111-05-06   
12  23                      耳鼻喉頭頸部契約語言治療師甄選公告  111-05-08   
13  25                護理部 大武院區儲備契約護理(手術專責護理師)  111-05-26   
14  27      心臟血管外科(屏東大武院區)-契約醫事技術師(體外循環師）甄選公告  111-06-30   
15  29                      大武院區儲備契約護理(專